## LangGraph Open Deep Research - Supervisor-Researcher Architecture

In this notebook, we'll explore the **supervisor-researcher delegation architecture** for conducting deep research with LangGraph.

You can visit this repository to see the original application: [Open Deep Research](https://github.com/langchain-ai/open_deep_research)

Let's jump in!

## What We're Building

This implementation uses a **hierarchical delegation pattern** where:

1. **User Clarification** - Optionally asks clarifying questions to understand the research scope
2. **Research Brief Generation** - Transforms user messages into a structured research brief
3. **Supervisor** - A lead researcher that analyzes the brief and delegates research tasks
4. **Parallel Researchers** - Multiple sub-agents that conduct focused research simultaneously
5. **Research Compression** - Each researcher synthesizes their findings
6. **Final Report** - All findings are combined into a comprehensive report

![Architecture Diagram](Open%20Deep%20Research.png)

This differs from a section-based approach by allowing dynamic task decomposition based on the research question, rather than predefined sections.

## Dependencies

You'll need API keys for Anthropic (for the LLM) and Tavily (for web search). We'll configure the system to use Anthropic's Claude Sonnet 4 exclusively.

In [59]:
import os
import getpass

os.environ["ANTHROPIC_API_KEY"] = getpass.getpass("Enter your Anthropic API key: ")
os.environ["TAVILY_API_KEY"] = getpass.getpass("Enter your Tavily API key: ")

## Task 1: State Definitions

The state structure is hierarchical with three levels:

### Agent State (Top Level)
Contains the overall conversation messages, research brief, accumulated notes, and final report.

### Supervisor State (Middle Level)
Manages the research supervisor's messages, research iterations, and coordinating parallel researchers.

### Researcher State (Bottom Level)
Each individual researcher has their own message history, tool call iterations, and research findings.

We also have structured outputs for tool calling:
- **ConductResearch** - Tool for supervisor to delegate research to a sub-agent
- **ResearchComplete** - Tool to signal research phase is done
- **ClarifyWithUser** - Structured output for asking clarifying questions
- **ResearchQuestion** - Structured output for the research brief

Let's import these from our library: [`open_deep_library/state.py`](open_deep_library/state.py)

In [60]:
# Import state definitions from the library
from open_deep_library.state import (
    # Main workflow states
    AgentState,           # Lines 65-72: Top-level agent state with messages, research_brief, notes, final_report
    AgentInputState,      # Lines 62-63: Input state is just messages
    
    # Supervisor states
    SupervisorState,      # Lines 74-81: Supervisor manages research delegation and iterations
    
    # Researcher states
    ResearcherState,      # Lines 83-90: Individual researcher with messages and tool iterations
    ResearcherOutputState, # Lines 92-96: Output from researcher (compressed research + raw notes)
    
    # Structured outputs for tool calling
    ConductResearch,      # Lines 15-19: Tool for delegating research to sub-agents
    ResearchComplete,     # Lines 21-22: Tool to signal research completion
    ClarifyWithUser,      # Lines 30-41: Structured output for user clarification
    ResearchQuestion,     # Lines 43-48: Structured output for research brief
)

#### ❓ Question 1:

 Explain the interrelationships between the three states.  Why don't we just make a single huge state?

##### ✅ Answer: 
The agent states interacts with the user and orchestrates overall workflow and final output. The supervisor state manages the research strategy and invokes the researches in parallel. The researcher state executes that focused research tasks independently and returns status. The benefits of this approach are separating parallel execution of researcher async processes,role clarity, ease of maintenance and trouble shooting to find errors. Also, when we have a single huge state it becomes difficult to understand which part is executed can make user anxious without a clue to the user what is being executed and having to wait until the final response. 

## Task 2: Utility Functions and Tools

The system uses several key utilities:

### Search Tools
- **tavily_search** - Async web search with automatic summarization to stay within token limits
- Supports Anthropic native web search and Tavily API

### Reflection Tools
- **think_tool** - Allows researchers to reflect on their progress and plan next steps (ReAct pattern)

### Helper Utilities
- **get_all_tools** - Assembles the complete toolkit (search + MCP + reflection)
- **get_today_str** - Provides current date context for research
- Token limit handling utilities for graceful degradation

These are defined in [`open_deep_library/utils.py`](open_deep_library/utils.py)

In [61]:
# Import utility functions and tools from the library
from open_deep_library.utils import (
    # Search tool - Lines 43-136: Tavily search with automatic summarization
    tavily_search,
    
    # Reflection tool - Lines 219-244: Strategic thinking tool for ReAct pattern
    think_tool,
    
    # Tool assembly - Lines 569-597: Get all configured tools
    get_all_tools,
    
    # Date utility - Lines 872-879: Get formatted current date
    get_today_str,
    
    # Supporting utilities for error handling
    get_api_key_for_model,          # Lines 892-914: Get API keys from config or env
    is_token_limit_exceeded,         # Lines 665-701: Detect token limit errors
    get_model_token_limit,           # Lines 831-846: Look up model's token limit
    remove_up_to_last_ai_message,    # Lines 848-866: Truncate messages for retry
    anthropic_websearch_called,      # Lines 607-637: Detect Anthropic native search usage
    openai_websearch_called,         # Lines 639-658: Detect OpenAI native search usage
    get_notes_from_tool_calls,       # Lines 599-601: Extract notes from tool messages
)

### ❓ Question 2:  

What are the advantages and disadvantages of importing these components instead of including them in the notebook?

##### ✅ Answer:
#### Advantages: 
1.Module and Clean code is easy to read and maintain

2.Easy to test, debug and modify single unit components

3.Production ready code enabling parallel development and deployment

4.Makes Utils reusable tools as packages

5.Adhering Software engineering best practices like SOLID & DRY

#### Disadvantages:
1.It is complex for beginners, there is a learning curve if not from software development

2.Bugs in the utils can impact every project that is used ( single point of failure)

3.Need to know dependency management for imports and version conflict errors



## Task 3: Configuration System

The configuration system controls:

### Research Behavior
- **allow_clarification** - Whether to ask clarifying questions before research
- **max_concurrent_research_units** - How many parallel researchers can run (default: 5)
- **max_researcher_iterations** - How many times supervisor can delegate research (default: 6)
- **max_react_tool_calls** - Tool call limit per researcher (default: 10)

### Model Configuration
- **research_model** - Model for research and supervision (we'll use Anthropic)
- **compression_model** - Model for synthesizing findings
- **final_report_model** - Model for writing the final report
- **summarization_model** - Model for summarizing web search results

### Search Configuration
- **search_api** - Which search API to use (ANTHROPIC, TAVILY, or NONE)
- **max_content_length** - Character limit before summarization

Defined in [`open_deep_library/configuration.py`](open_deep_library/configuration.py)

In [62]:
# Import configuration from the library
from open_deep_library.configuration import (
    Configuration,    # Lines 38-247: Main configuration class with all settings
    SearchAPI,        # Lines 11-17: Enum for search API options (ANTHROPIC, TAVILY, NONE)
)

## Task 4: Prompt Templates

The system uses carefully engineered prompts for each phase:

### Phase 1: Clarification
**clarify_with_user_instructions** - Analyzes if the research scope is clear or needs clarification

### Phase 2: Research Brief
**transform_messages_into_research_topic_prompt** - Converts user messages into a detailed research brief

### Phase 3: Supervisor
**lead_researcher_prompt** - System prompt for the supervisor that manages delegation strategy

### Phase 4: Researcher
**research_system_prompt** - System prompt for individual researchers conducting focused research

### Phase 5: Compression
**compress_research_system_prompt** - Prompt for synthesizing research findings without losing information

### Phase 6: Final Report
**final_report_generation_prompt** - Comprehensive prompt for writing the final report

All prompts are defined in [`open_deep_library/prompts.py`](open_deep_library/prompts.py)

In [63]:
# Import prompt templates from the library
from open_deep_library.prompts import (
    clarify_with_user_instructions,                    # Lines 3-41: Ask clarifying questions
    transform_messages_into_research_topic_prompt,     # Lines 44-77: Generate research brief
    lead_researcher_prompt,                            # Lines 79-136: Supervisor system prompt
    research_system_prompt,                            # Lines 138-183: Researcher system prompt
    compress_research_system_prompt,                   # Lines 186-222: Research compression prompt
    final_report_generation_prompt,                    # Lines 228-308: Final report generation
)

## Task 5: Node Functions - The Building Blocks

Now let's look at the node functions that make up our graph. We'll import them from the library and understand what each does.

### The Complete Research Workflow

The workflow consists of 8 key nodes organized into 3 subgraphs:

1. **Main Graph Nodes:**
   - `clarify_with_user` - Entry point that checks if clarification is needed
   - `write_research_brief` - Transforms user input into structured research brief
   - `final_report_generation` - Synthesizes all research into final report

2. **Supervisor Subgraph Nodes:**
   - `supervisor` - Lead researcher that plans and delegates
   - `supervisor_tools` - Executes supervisor's tool calls (delegation, reflection)

3. **Researcher Subgraph Nodes:**
   - `researcher` - Individual researcher conducting focused research
   - `researcher_tools` - Executes researcher's tool calls (search, reflection)
   - `compress_research` - Synthesizes researcher's findings

All nodes are defined in [`open_deep_library/deep_researcher.py`](open_deep_library/deep_researcher.py)

### Node 1: clarify_with_user

**Purpose:** Analyzes user messages and asks clarifying questions if the research scope is unclear.

**Key Steps:**
1. Check if clarification is enabled in configuration
2. Use structured output to analyze if clarification is needed
3. If needed, end with a clarifying question for the user
4. If not needed, proceed to research brief with verification message

**Implementation:** [`open_deep_library/deep_researcher.py` lines 60-115](open_deep_library/deep_researcher.py#L60-L115)

In [64]:
# Import the clarify_with_user node
from open_deep_library.deep_researcher import clarify_with_user

### Node 2: write_research_brief

**Purpose:** Transforms user messages into a structured research brief for the supervisor.

**Key Steps:**
1. Use structured output to generate detailed research brief from messages
2. Initialize supervisor with system prompt and research brief
3. Set up supervisor messages with proper context

**Why this matters:** A well-structured research brief helps the supervisor make better delegation decisions.

**Implementation:** [`open_deep_library/deep_researcher.py` lines 118-175](open_deep_library/deep_researcher.py#L118-L175)

In [65]:
# Import the write_research_brief node
from open_deep_library.deep_researcher import write_research_brief

### Node 3: supervisor

**Purpose:** Lead research supervisor that plans research strategy and delegates to sub-researchers.

**Key Steps:**
1. Configure model with three tools:
   - `ConductResearch` - Delegate research to a sub-agent
   - `ResearchComplete` - Signal that research is done
   - `think_tool` - Strategic reflection before decisions
2. Generate response based on current context
3. Increment research iteration count
4. Proceed to tool execution

**Decision Making:** The supervisor uses `think_tool` to reflect before delegating research, ensuring thoughtful decomposition of the research question.

**Implementation:** [`open_deep_library/deep_researcher.py` lines 178-223](open_deep_library/deep_researcher.py#L178-L223)

In [66]:
# Import the supervisor node (from supervisor subgraph)
from open_deep_library.deep_researcher import supervisor

### Node 4: supervisor_tools

**Purpose:** Executes the supervisor's tool calls, including strategic thinking and research delegation.

**Key Steps:**
1. Check exit conditions:
   - Exceeded maximum iterations
   - No tool calls made
   - `ResearchComplete` called
2. Process `think_tool` calls for strategic reflection
3. Execute `ConductResearch` calls in parallel:
   - Spawn researcher subgraphs for each delegation
   - Limit to `max_concurrent_research_units` (default: 5)
   - Gather all results asynchronously
4. Aggregate findings and return to supervisor

**Parallel Execution:** This is where the magic happens - multiple researchers work simultaneously on different aspects of the research question.

**Implementation:** [`open_deep_library/deep_researcher.py` lines 225-349](open_deep_library/deep_researcher.py#L225-L349)

In [67]:
# Import the supervisor_tools node
from open_deep_library.deep_researcher import supervisor_tools

### Node 5: researcher

**Purpose:** Individual researcher that conducts focused research on a specific topic.

**Key Steps:**
1. Load all available tools (search, MCP, reflection)
2. Configure model with tools and researcher system prompt
3. Generate response with tool calls
4. Increment tool call iteration count

**ReAct Pattern:** Researchers use `think_tool` to reflect after each search, deciding whether to continue or provide their answer.

**Available Tools:**
- Search tools (Tavily or Anthropic native search)
- `think_tool` for strategic reflection
- `ResearchComplete` to signal completion
- MCP tools (if configured)

**Implementation:** [`open_deep_library/deep_researcher.py` lines 365-424](open_deep_library/deep_researcher.py#L365-L424)

In [68]:
# Import the researcher node (from researcher subgraph)
from open_deep_library.deep_researcher import researcher

### Node 6: researcher_tools

**Purpose:** Executes the researcher's tool calls, including searches and strategic reflection.

**Key Steps:**
1. Check early exit conditions (no tool calls, native search used)
2. Execute all tool calls in parallel:
   - Search tools fetch and summarize web content
   - `think_tool` records strategic reflections
   - MCP tools execute external integrations
3. Check late exit conditions:
   - Exceeded `max_react_tool_calls` (default: 10)
   - `ResearchComplete` called
4. Continue research loop or proceed to compression

**Error Handling:** Safely handles tool execution errors and continues with available results.

**Implementation:** [`open_deep_library/deep_researcher.py` lines 435-509](open_deep_library/deep_researcher.py#L435-L509)

In [69]:
# Import the researcher_tools node
from open_deep_library.deep_researcher import researcher_tools

### Node 7: compress_research

**Purpose:** Compresses and synthesizes research findings into a concise, structured summary.

**Key Steps:**
1. Configure compression model
2. Add compression instruction to messages
3. Attempt compression with retry logic:
   - If token limit exceeded, remove older messages
   - Retry up to 3 times
4. Extract raw notes from tool and AI messages
5. Return compressed research and raw notes

**Why Compression?** Researchers may accumulate lots of tool outputs and reflections. Compression ensures:
- All important information is preserved
- Redundant information is deduplicated
- Content stays within token limits for the final report

**Token Limit Handling:** Gracefully handles token limit errors by progressively truncating messages.

**Implementation:** [`open_deep_library/deep_researcher.py` lines 511-585](open_deep_library/deep_researcher.py#L511-L585)

In [70]:
# Import the compress_research node
from open_deep_library.deep_researcher import compress_research

### Node 8: final_report_generation

**Purpose:** Generates the final comprehensive research report from all collected findings.

**Key Steps:**
1. Extract all notes from completed research
2. Configure final report model
3. Attempt report generation with retry logic:
   - If token limit exceeded, truncate findings by 10%
   - Retry up to 3 times
4. Return final report or error message

**Token Limit Strategy:**
- First retry: Use model's token limit × 4 as character limit
- Subsequent retries: Reduce by 10% each time
- Graceful degradation with helpful error messages

**Report Quality:** The prompt guides the model to create well-structured reports with:
- Proper headings and sections
- Inline citations
- Comprehensive coverage of all findings
- Sources section at the end

**Implementation:** [`open_deep_library/deep_researcher.py` lines 607-697](open_deep_library/deep_researcher.py#L607-L697)

In [71]:
# Import the final_report_generation node
from open_deep_library.deep_researcher import final_report_generation

## Task 6: Graph Construction - Putting It All Together

The system is organized into three interconnected graphs:

### 1. Researcher Subgraph (Bottom Level)
Handles individual focused research on a specific topic:
```
START → researcher → researcher_tools → compress_research → END
               ↑            ↓
               └────────────┘ (loops until max iterations or ResearchComplete)
```

### 2. Supervisor Subgraph (Middle Level)
Manages research delegation and coordination:
```
START → supervisor → supervisor_tools → END
            ↑              ↓
            └──────────────┘ (loops until max iterations or ResearchComplete)
            
supervisor_tools spawns multiple researcher_subgraphs in parallel
```

### 3. Main Deep Researcher Graph (Top Level)
Orchestrates the complete research workflow:
```
START → clarify_with_user → write_research_brief → research_supervisor → final_report_generation → END
                 ↓                                       (supervisor_subgraph)
               (may end early if clarification needed)
```

Let's import the compiled graphs from the library.

In [72]:
# Import the pre-compiled graphs from the library
from open_deep_library.deep_researcher import (
    # Bottom level: Individual researcher workflow
    researcher_subgraph,    # Lines 588-605: researcher → researcher_tools → compress_research
    
    # Middle level: Supervisor coordination
    supervisor_subgraph,    # Lines 351-363: supervisor → supervisor_tools (spawns researchers)
    
    # Top level: Complete research workflow
    deep_researcher,        # Lines 699-719: Main graph with all phases
)

## Why This Architecture?

### Advantages of Supervisor-Researcher Delegation

1. **Dynamic Task Decomposition**
   - Unlike section-based approaches with predefined structure, the supervisor can break down research based on the actual question
   - Adapts to different types of research (comparisons, lists, deep dives, etc.)

2. **Parallel Execution**
   - Multiple researchers work simultaneously on different aspects
   - Much faster than sequential section processing
   - Configurable parallelism (1-20 concurrent researchers)

3. **ReAct Pattern for Quality**
   - Researchers use `think_tool` to reflect after each search
   - Prevents excessive searching and improves search quality
   - Natural stopping conditions based on information sufficiency

4. **Flexible Tool Integration**
   - Easy to add MCP tools for specialized research
   - Supports multiple search APIs (Anthropic, Tavily)
   - Each researcher can use different tool combinations

5. **Graceful Token Limit Handling**
   - Compression prevents token overflow
   - Progressive truncation in final report generation
   - Research can scale to arbitrary depths

### Trade-offs

- **Complexity:** More moving parts than section-based approach
- **Cost:** Parallel researchers use more tokens (but faster)
- **Unpredictability:** Research structure emerges dynamically

## Task 7: Running the Deep Researcher

Now let's see the system in action! We'll use it to analyze a PDF document about how people use AI.

### Setup

We need to:
1. Load the PDF document
2. Configure the execution with Anthropic settings
3. Run the research workflow

In [73]:
# Load the PDF document
from pathlib import Path
import PyPDF2

def load_pdf(pdf_path: str) -> str:
    """Load and extract text from PDF."""
    pdf_text = ""
    with open(pdf_path, 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)
        for page in pdf_reader.pages:
            pdf_text += page.extract_text() + "\n\n"
    return pdf_text

# Load the PDF about how people use AI
pdf_path = "data/howpeopleuseai.pdf"
pdf_content = load_pdf(pdf_path)

print(f"Loaded PDF with {len(pdf_content)} characters")
print(f"First 500 characters:\n{pdf_content[:500]}...")

Loaded PDF with 112460 characters
First 500 characters:
NBER WORKING PAPER SERIES
HOW PEOPLE USE CHATGPT
Aaron Chatterji
Thomas Cunningham
David J. Deming
Zoe Hitzig
Christopher Ong
Carl Yan Shan
Kevin Wadman
Working Paper 34255
http://www.nber.org/papers/w34255
NATIONAL BUREAU OF ECONOMIC RESEARCH
1050 Massachusetts Avenue
Cambridge, MA 02138
September 2025
We acknowledge help and comments from Joshua Achiam, Hemanth Asirvatham, Ryan 
Beiermeister,  Rachel Brown, Cassandra Duchan Solis, Jason Kwon, Elliott Mokski, Kevin Rao, 
Harrison Satcher,  Gawe...


In [74]:
# Set up the graph with Anthropic configuration
from IPython.display import Markdown, display
import uuid

# Note: deep_researcher is already compiled from the library
# For this demo, we'll use it directly without additional checkpointing
graph = deep_researcher

print("✓ Graph ready for execution")
print("  (Note: The graph is pre-compiled from the library)")

✓ Graph ready for execution
  (Note: The graph is pre-compiled from the library)


### Configuration for Anthropic

We'll configure the system to use:
- **Claude Sonnet 4** for all research, supervision, and report generation
- **Tavily** for web search (you can also use Anthropic's native search)
- **Moderate parallelism** (3 concurrent researchers)
- **Clarification enabled** (will ask if research scope is unclear)

In [75]:
# Configure for Anthropic with moderate settings
config = {
    "configurable": {
        # Model configuration - using Claude Sonnet 4 for everything
        "research_model": "anthropic:claude-sonnet-4-20250514",
        "research_model_max_tokens": 10000,
        
        "compression_model": "anthropic:claude-sonnet-4-20250514",
        "compression_model_max_tokens": 8192,
        
        "final_report_model": "anthropic:claude-sonnet-4-20250514",
        "final_report_model_max_tokens": 10000,
        
        "summarization_model": "anthropic:claude-sonnet-4-20250514",
        "summarization_model_max_tokens": 8192,
        
        # Research behavior
        "allow_clarification": True,
        "max_concurrent_research_units": 1,  # 1 parallel researchers
        "max_researcher_iterations": 2,      # Supervisor can delegate up to 2 times
        "max_react_tool_calls": 3,           # Each researcher can make up to 3 tool calls
        
        # Search configuration
        "search_api": "tavily",  # Using Tavily for web search
        "max_content_length": 50000,
        
        # Thread ID for this conversation
        "thread_id": str(uuid.uuid4())
    }
}

print("✓ Configuration ready")
print(f"  - Research Model: Claude Sonnet 4")
print(f"  - Max Concurrent Researchers: 3")
print(f"  - Max Iterations: 4")
print(f"  - Search API: Tavily")

✓ Configuration ready
  - Research Model: Claude Sonnet 4
  - Max Concurrent Researchers: 3
  - Max Iterations: 4
  - Search API: Tavily


### Execute the Research

Now let's run the research! We'll ask the system to analyze the PDF and provide insights about how people use AI.

The workflow will:
1. **Clarify** - Check if the request is clear (may skip if obvious)
2. **Research Brief** - Transform our request into a structured brief
3. **Supervisor** - Plan research strategy and delegate to researchers
4. **Parallel Research** - Multiple researchers gather information simultaneously
5. **Compression** - Each researcher synthesizes their findings
6. **Final Report** - All findings combined into comprehensive report

In [76]:
# Create our research request with PDF context
research_request = f"""
I have a PDF document about how people use AI. Please analyze this document and provide insights about:

1. What are the main findings about how people are using AI?
2. What are the most common use cases?
3. What trends or patterns emerge from the data?

Here's the PDF content:

{pdf_content[:10000]}  # First 10k chars to stay within limits

...[content truncated for context window]
"""

# Execute the graph
async def run_research():
    """Run the research workflow and display results."""
    print("Starting research workflow...\n")
    
    async for event in graph.astream(
        {"messages": [{"role": "user", "content": research_request}]},
        config,
        stream_mode="updates"
    ):
        # Display each step
        for node_name, node_output in event.items():
            print(f"\n{'='*60}")
            print(f"Node: {node_name}")
            print(f"{'='*60}")
            
            if node_name == "clarify_with_user":
                if "messages" in node_output:
                    last_msg = node_output["messages"][-1]
                    print(f"\n{last_msg.content}")
            
            elif node_name == "write_research_brief":
                if "research_brief" in node_output:
                    print(f"\nResearch Brief Generated:")
                    print(f"{node_output['research_brief'][:500]}...")
            
            elif node_name == "supervisor":
                print(f"\nSupervisor planning research strategy...")
                if "supervisor_messages" in node_output:
                    last_msg = node_output["supervisor_messages"][-1]
                    if hasattr(last_msg, 'tool_calls') and last_msg.tool_calls:
                        print(f"Tool calls: {len(last_msg.tool_calls)}")
                        for tc in last_msg.tool_calls:
                            print(f"  - {tc['name']}")
            
            elif node_name == "supervisor_tools":
                print(f"\nExecuting supervisor's tool calls...")
                if "notes" in node_output:
                    print(f"Research notes collected: {len(node_output['notes'])}")
            
            elif node_name == "final_report_generation":
                if "final_report" in node_output:
                    print(f"\n" + "="*60)
                    print("FINAL REPORT GENERATED")
                    print("="*60 + "\n")
                    display(Markdown(node_output["final_report"]))
    
    print("\n" + "="*60)
    print("Research workflow completed!")
    print("="*60)

# Run the research
await run_research()

Starting research workflow...


Node: clarify_with_user

I have sufficient information to proceed with analyzing the NBER working paper "How People Use ChatGPT." You've provided the PDF content and requested specific insights about:

1. Main findings about how people are using AI
2. Most common use cases 
3. Trends and patterns emerging from the data

Based on the document provided, I can see this is a comprehensive study covering ChatGPT usage from November 2022 through July 2025, with detailed analysis of work vs. non-work usage, conversation topics, and user demographics. I will now begin analyzing this research paper to extract the key insights you requested.

Node: write_research_brief

Research Brief Generated:
I have a PDF document containing the NBER Working Paper "How People Use ChatGPT" by Aaron Chatterji, Thomas Cunningham, David J. Deming, Zoe Hitzig, Christopher Ong, Carl Yan Shan, and Kevin Wadman (Working Paper 34255, September 2025). I need you to analyze this specific 


Node: research_supervisor

Node: final_report_generation

FINAL REPORT GENERATED



# How People Use ChatGPT: Comprehensive Analysis of NBER Working Paper 34255

## Introduction and Study Scope

The NBER Working Paper 34255 titled "How People Use ChatGPT" represents the most comprehensive study to date on ChatGPT usage patterns, conducted by researchers from OpenAI, Duke University, and Harvard University. Published in September 2025, this groundbreaking research documents ChatGPT's unprecedented growth from its November 2022 launch through July 2025, when it had been adopted by approximately 10% of the world's adult population—roughly 700 million users processing 18 billion messages weekly [1][3].

The study's scale is remarkable: by June 2025, ChatGPT was processing more than 2.6 billion messages per day, or over 30,000 messages per second [1]. This represents a 5.8x increase in total message volume over just one year, while user growth was 3.2x, indicating that users are engaging more intensively with the technology as they gain experience [1].

## Unprecedented Adoption and Growth Patterns

ChatGPT achieved the fastest application adoption in history, reaching 1 million users in just 5 days after launch and 100 million weekly active users within less than one year [1][2]. The platform's growth has been exponential, with weekly active users doubling every 7-8 months and reaching over 750 million users by September 2025 [1].

The research reveals that all user cohorts have increased their usage significantly, particularly beginning in late 2024 and early 2025. Even the earliest adopters from Q1 2023, after experiencing some decline through 2024, were sending 40% more messages per day by July 2025 than they did two years earlier. More recent users who signed up in Q3 and Q4 2024 nearly doubled their daily message volume within less than a year [1].

## Work vs. Non-Work Usage Evolution

One of the study's most significant findings concerns the dramatic shift in how ChatGPT is used. While both work-related and non-work-related messages have grown continuously, non-work usage has expanded much faster. The data shows a clear trend: non-work-related messages increased from 53% in June 2024 to 73% by June 2025, while work-related usage correspondingly decreased from 47% to 27% over the same period [1][3][6].

This shift is primarily driven by changing usage patterns within existing user cohorts rather than changes in the composition of new users [6]. The finding challenges common assumptions about AI's primary value proposition, suggesting that while economic analysis has focused heavily on productivity gains in paid work, the impact on non-work activities (home production) may be on a similar or even larger scale [6].

Work usage remains more prevalent among educated users in highly-paid professional occupations, but the overall trend indicates ChatGPT's expanding role in personal and recreational activities [2][3][6].

## Most Common Use Cases: The Big Three

The research identifies that nearly 80% of all ChatGPT usage falls into three dominant categories, representing the core value propositions of the platform [2][3][6][8]:

### Practical Guidance (29% of usage)
This is the single most common use case, maintaining consistent usage at roughly 29% of all interactions. Practical Guidance encompasses tutoring and teaching activities (10.2% of all messages), how-to advice on various topics (8.5% of all messages), and creative ideation. Education emerges as a major application, with 36% of Practical Guidance messages being requests for tutoring or teaching, while another 30% consists of general how-to advice [6][8].

### Seeking Information (24% of usage, up from 14% in July 2024)
This category has shown remarkable growth and includes searching for information about people, current events, products, and recipes. The research suggests this represents a very close substitute for traditional web search engines, indicating ChatGPT's growing role in information discovery and retrieval [2][3][6][9].

### Writing (24% of usage, down from 36% in July 2024)
Despite some decline in relative share, Writing remains a major use case encompassing automated production of emails, documents, and communications, as well as editing, critiquing, summarizing, and translating existing text. Notably, about two-thirds of all Writing messages involve modifying user-provided text rather than creating entirely new content from scratch [2]. Writing dominates work-related tasks, accounting for 40% of work-related messages in June 2025 [2][6][8].

## User Intent Classifications: Asking, Doing, Expressing

The study introduces an innovative taxonomy classifying messages by user intent, revealing important patterns in how people interact with AI:

**Asking (49% of messages)** represents users seeking information or clarification to inform decisions, corresponding to problem-solving models of knowledge work. These messages consistently receive higher quality ratings from both automated classifiers and direct user feedback [6].

**Doing (40% of messages)** involves users wanting to produce specific outputs or perform particular tasks, corresponding to traditional task-based work models. For work-related messages specifically, about 56% are classified as Doing, with nearly three-quarters of those being Writing tasks [6].

**Expressing (11% of messages)** encompasses users expressing views or feelings without seeking information or action. This category has grown from under 8% in July 2024 to 13.8% by June 2025, indicating increasing use of ChatGPT for social-emotional purposes [6].

## Demographic Trends and Shifts

### Gender Demographics
The research documents a dramatic narrowing of the gender gap in ChatGPT usage. Early adopters were heavily male-skewed, with about 80% having typically masculine first names in the initial months. However, by June 2025, this had reversed to 48% masculine and 52% feminine first names, indicating successful adoption across gender lines [1][6][7][10].

### Age Demographics
Young users dominate ChatGPT usage, with nearly half of all adult messages sent by users under 26, and more than 45% of all users being under 25 [2][6][10]. However, age-related usage patterns vary significantly: work-related messages comprised only 23% for users under 26, increasing with age, though users aged 66 or older sent just 16% of their messages for work purposes [12].

### Geographic Growth
The study reveals particularly strong growth in low- and middle-income countries, with usage increasing by 5-6x in middle-income countries compared to 3x growth in the richest countries [1][9][10]. This geographic diversification suggests ChatGPT is becoming a global tool rather than remaining concentrated in wealthy nations.

## Programming and Technical Usage: Lower Than Expected

Contrary to widespread assumptions about AI's primary use in technical fields, computer programming accounts for only 4.2% of all ChatGPT messages [6][8][11]. This is significantly lower than other platforms like Claude, where programming represents 33% of work-related conversations [6][11]. 

Technical Help overall, including programming (4.2%), mathematical calculations (3%), and data analysis (0.4%), has actually declined from 12% of all usage in July 2024 to around 5% a year later [6]. This finding challenges the tech community's focus on coding capabilities and suggests broader, more diverse applications drive adoption [11].

## Workplace Integration and O*NET Mapping

The research mapped ChatGPT usage to workplace activities using the Occupational Information Network (O*NET), revealing that about 58% of work-related messages are associated with two broad categories: obtaining, documenting, and interpreting information; and making decisions, giving advice, solving problems, and thinking creatively [6].

Remarkably, the work activities associated with ChatGPT usage show consistency across very different occupations. "Getting Information" and "Making Decisions and Solving Problems" rank in the top five message frequencies across management, business, STEM, administrative, and sales occupations [6][12]. This suggests ChatGPT's value transcends specific professional domains, providing universal support for knowledge work activities.

## Social-Emotional Usage: Smaller Than Expected

Despite speculation about AI companionship, the research finds relatively limited usage for social-emotional purposes. Only 1.9% of messages relate to relationships and personal reflection, while just 0.4% involve games and role-play [6]. This contrasts sharply with other estimates suggesting therapy and companionship were prevalent use cases for generative AI [6].

## Economic Value and Decision Support

The study concludes that ChatGPT's primary economic value lies in decision support, particularly important in knowledge-intensive jobs [2][6]. Rather than simply automating tasks, ChatGPT serves as a decision-support system, helping users weigh options, choose better words, and interpret information [8].

The research indicates the highest satisfaction and fastest growth occur in advisory roles such as tutoring, giving advice, and helping people think through problems rather than pure task completion [13]. This aligns with the finding that almost half of all usage falls into Practical Guidance or Seeking Information categories [6].

## Quality and User Satisfaction Patterns

"Asking" messages consistently receive higher quality ratings compared to "Doing" messages, both from automated satisfaction classifiers and direct user feedback [6]. This suggests users find ChatGPT most valuable when seeking guidance and information rather than task execution, reinforcing the platform's strength as an advisory tool rather than a pure automation solution.

## Implications for AI Development and Society

The research reveals ChatGPT's evolution from a primarily work-focused tool to a broader platform supporting diverse personal and professional needs. The dominance of non-work usage (73% by June 2025) suggests the economic impact of AI extends far beyond workplace productivity into home production and personal development.

The study's finding that information-seeking has grown dramatically (from 14% to 24% of usage) indicates ChatGPT is increasingly competing with traditional search engines, while the consistent strength of practical guidance suggests unique value in advisory and educational contexts that traditional search cannot provide.

## Methodology and Privacy Protection

The research employed rigorous privacy-preserving methodologies, with no research team member ever viewing actual user message content. Instead, automated classifiers analyzed messages stripped of personally identifiable information through an internal tool called Privacy Filter, with all analysis conducted using Data Clean Rooms and aggregated outputs over limited categories [1][6]. This approach enables comprehensive usage analysis while maintaining user privacy—a critical consideration for studying sensitive communication data.

### Sources

[1] How People Use ChatGPT - by David Deming - Forked Lightning: https://forklightning.substack.com/p/how-people-use-chatgpt

[2] Number of ChatGPT Users (October 2025) - Exploding Topics: https://explodingtopics.com/blog/chatgpt-users

[3] 73% of ChatGPT Uses Are Non-Work Related, OpenAI Study Finds: https://tech.co/news/chatgpt-uses-non-work-related-openai-study

[4] How People Are Really Using ChatGPT - Mike Jeffs: https://mikejeffs.com/blog/how-people-are-really-using-chatgpt/

[5] How People Actually Use ChatGPT — What 1.5M Conversations Tell Us: https://medium.com/@adnanmasood/how-people-actually-use-chatgpt-what-1-5m-conversations-tell-us-about-the-next-decade-of-software-ea603212b458

[6] How People Use ChatGPT | OpenAI (PDF): https://cdn.openai.com/pdf/a253471f-8260-40c6-a2cc-aa93fe9f142e/economic-research-chatgpt-usage-paper.pdf

[7] How People Use ChatGPT - Edward Conard: https://www.edwardconard.com/macro-roundup/as-of-july-2025-chatgpt-was-used-at-least-weekly-by-10-of-the-global-adult-population-70-of-usage-was-non-work-related-work-usage-was-most-common-for-educated-users-in-high-pay-professional-occupa/?view=detail

[8] ChatGPT usage: 80% of conversations fall into three categories: https://www.linkedin.com/posts/aaron-ronnie-chatterji_nearly-80-of-all-chatgpt-conversations-fall-activity-7374916815124131841-VSwt

[9] ChatGPT Study: 1 In 4 Conversations Now Seek Information: https://www.searchenginejournal.com/chatgpt-study-1-in-4-conversations-now-seek-information/556104/

[10] ChatGPT Study Reveals 1 in 4 Conversations Now Seek Information: https://www.ravi-gupta.com/news/chatgpt-study-1-in-4-conversations-now-seek-information/

[11] How people use ChatGPT: NBER study reveals usage patterns: https://www.linkedin.com/posts/julienchaumond_chatgpt-usage-breakdown-aka-how-activity-7374435587233329153-yYBp

[12] Over 70% ChatGPT Interactions Are Non-Work, Says OpenAI: https://www.medianama.com/2025/09/223-over-70-chatgpt-interactions-non-work-guidance-openai/

[13] The Impact of ChatGPT Exposure on User Interactions: https://pmc.ncbi.nlm.nih.gov/articles/PMC11952273/


Research workflow completed!


## Understanding the Output

Let's break down what happened:

### Phase 1: Clarification
The system checked if your request was clear. Since you provided a PDF and specific questions, it likely proceeded without clarification.

### Phase 2: Research Brief
Your request was transformed into a detailed research brief that guides the supervisor's delegation strategy.

### Phase 3: Supervisor Delegation
The supervisor analyzed the brief and decided how to break down the research:
- Used `think_tool` to plan strategy
- Called `ConductResearch` multiple times to delegate to parallel researchers
- Each delegation specified a focused research topic

### Phase 4: Parallel Research
Multiple researchers worked simultaneously:
- Each researcher used web search tools to gather information
- Used `think_tool` to reflect after each search
- Decided when they had enough information
- Compressed their findings into clean summaries

### Phase 5: Final Report
All research findings were synthesized into a comprehensive report with:
- Well-structured sections
- Inline citations
- Sources listed at the end
- Balanced coverage of all findings

#### 🏗️ Activity #1: Try Different Configurations

You can experiment with different settings to see how they affect the research.  You may select three or more of the following settings (or invent your own experiments) and describe the results.

### Increase Parallelism
```python
"max_concurrent_research_units": 10  # More researchers working simultaneously
```

### Deeper Research
```python
"max_researcher_iterations": 8   # Supervisor can delegate more times
"max_react_tool_calls": 15      # Each researcher can search more
```

### Use Anthropic Native Search
```python
"search_api": "anthropic"  # Use Claude's built-in web search
```

### Disable Clarification
```python
"allow_clarification": False  # Skip clarification phase
```

In [84]:
# Configure for Anthropic with moderate settings
config = {
    "configurable": {
        # Model configuration - using Claude Sonnet 4 for everything
        "research_model": "anthropic:claude-sonnet-4-20250514",
        "research_model_max_tokens": 5000,
        
        "compression_model": "anthropic:claude-sonnet-4-20250514",
        "compression_model_max_tokens": 8192,
        
        "final_report_model": "anthropic:claude-sonnet-4-20250514",
        "final_report_model_max_tokens": 10000,
        
        "summarization_model": "anthropic:claude-sonnet-4-20250514",
        "summarization_model_max_tokens": 8192,
        
        # Research behavior
        "allow_clarification": True,  # Tried changing from True to False; got error TypeError: argument of type 'NoneType' is not iterable
        "max_concurrent_research_units": 1,  # 1 parallel researchers 
        "max_researcher_iterations": 2,      # Supervisor can delegate up to 2 times
        "max_react_tool_calls": 5,           # Each researcher can make up to 5 tool calls
        
        # Search configuration
        "search_api": "anthropic",  # Use Claude's built-in web search
        "max_content_length": 5000,
        
        # Thread ID for this conversation
        "thread_id": str(uuid.uuid4())
    }
}

print("✓ Configuration ready")
print(f"  - Research Model: Claude Sonnet 4")
print(f"  - Max Concurrent Researchers: 1")
print(f"  - Max Iterations: 2")
print(f"  - Search API: Anthropic")


✓ Configuration ready
  - Research Model: Claude Sonnet 4
  - Max Concurrent Researchers: 1
  - Max Iterations: 2
  - Search API: Anthropic


In [85]:
# Create our research request with PDF context
research_request = f"""
I have a PDF document about how people use AI. Please analyze this document and provide insights about:

1. What are the main findings about how people are using AI?
2. What are the most common use cases?
3. What trends or patterns emerge from the data?

Here's the PDF content:

{pdf_content[:2000]}  # First 2k chars to stay within limits

...[content truncated for context window]
"""

# Execute the graph
async def run_research():
    """Run the research workflow and display results."""
    print("Starting research workflow...\n")
    
    async for event in graph.astream(
        {"messages": [{"role": "user", "content": research_request}]},
        config,
        stream_mode="updates"
    ):
        # Display each step
        for node_name, node_output in event.items():
            print(f"\n{'='*60}")
            print(f"Node: {node_name}")
            print(f"{'='*60}")
            
            if node_name == "clarify_with_user":
                if "messages" in node_output:
                    last_msg = node_output["messages"][-1]
                    print(f"\n{last_msg.content}")
            
            elif node_name == "write_research_brief":
                if "research_brief" in node_output:
                    print(f"\nResearch Brief Generated:")
                    print(f"{node_output['research_brief'][:500]}...")
            
            elif node_name == "supervisor":
                print(f"\nSupervisor planning research strategy...")
                if "supervisor_messages" in node_output:
                    last_msg = node_output["supervisor_messages"][-1]
                    if hasattr(last_msg, 'tool_calls') and last_msg.tool_calls:
                        print(f"Tool calls: {len(last_msg.tool_calls)}")
                        for tc in last_msg.tool_calls:
                            print(f"  - {tc['name']}")
            
            elif node_name == "supervisor_tools":
                print(f"\nExecuting supervisor's tool calls...")
                if "notes" in node_output:
                    print(f"Research notes collected: {len(node_output['notes'])}")
            
            elif node_name == "final_report_generation":
                if "final_report" in node_output:
                    print(f"\n" + "="*60)
                    print("FINAL REPORT GENERATED")
                    print("="*60 + "\n")
                    display(Markdown(node_output["final_report"]))
    
    print("\n" + "="*60)
    print("Research workflow completed!")
    print("="*60)

# Run the research
await run_research()

Starting research workflow...


Node: clarify_with_user

I have sufficient information to proceed with analyzing the NBER working paper "How People Use ChatGPT" by Chatterji et al. (2025). I understand you want me to focus on three key areas: (1) main findings about how people are using AI, (2) the most common use cases, and (3) trends or patterns that emerge from the data. I will now begin analyzing the provided PDF content to extract these insights and provide you with a comprehensive report on ChatGPT usage patterns.

Node: write_research_brief

Research Brief Generated:
I need you to analyze the NBER working paper "How People Use ChatGPT" by Aaron Chatterji, Thomas Cunningham, David J. Deming, Zoe Hitzig, Christopher Ong, Carl Yan Shan, and Kevin Wadman (Working Paper No. 34255, September 2025) and provide comprehensive insights on three specific dimensions: (1) What are the main findings about how people are using AI/ChatGPT based on this research, (2) What are the most common use

# How People Use ChatGPT: Comprehensive Analysis of NBER Research Findings

The NBER working paper "How People Use ChatGPT" by Chatterji, Cunningham, Deming, Hitzig, Ong, Shan, and Wadman represents the first large-scale empirical study examining how people actually use AI chatbots in practice. This groundbreaking research provides unprecedented insights into user behavior patterns, usage trends, and the evolving role of AI in both work and personal contexts.

## Overview of Research Scope and Methodology

The study analyzed ChatGPT usage patterns from May 2024 to July 2025, examining 1.58 million messages from users through a privacy-preserving automated pipeline [1][2]. The research team employed multiple LLM-based classifiers to categorize messages while ensuring no human ever read the actual content of user messages [2][6]. The study was approved by Harvard IRB and utilized a secure data clean room protocol to protect user privacy while enabling comprehensive analysis [5][6].

The research focused exclusively on consumer plans (Free, Plus, and Pro), excluding business and enterprise accounts, and covered messages from users who had not opted out of data sharing [2][6]. This represents one of the most comprehensive analyses of AI chatbot usage patterns to date.

## Main Findings About AI/ChatGPT Usage Patterns

### Unprecedented Growth and Global Adoption

ChatGPT has achieved the fastest global technology diffusion in recorded history. Launched as a "research preview" on November 30, 2022, it reached one million registered users within five days [5]. By July 2025, ChatGPT had grown to serve 700 million users sending 18 billion messages weekly, representing approximately 10% of the global adult population [1][2][3].

The scale of this adoption is staggering when compared to other major technologies. ChatGPT reached 100 million weekly active users in less than one year, and by September 2025, had grown to more than 750 million weekly active users [5]. The number of users has been doubling every 7-8 months, while message volume has grown even faster at 5.8x in the last year compared to 3.2x growth in users [5].

### Shift from Work to Personal Use

One of the most significant findings concerns the evolution of how people use ChatGPT. The research reveals a dramatic shift from work-related to personal applications over time. Non-work messages grew from 53% in mid-2024 to more than 70% by mid-2025 [1][4][8]. This trend represents a fundamental change in user behavior patterns.

Critically, this shift is not primarily due to new types of users joining the platform, but rather reflects changing usage patterns within existing user cohorts [2]. As users become more familiar with ChatGPT's capabilities, they increasingly apply it to personal tasks and home production activities rather than just work-related functions.

### Economic Impact Beyond Workplace Productivity

The researchers emphasize that while most economic analysis of AI has focused on workplace productivity, the impact on activities outside of work appears to be on a similar or possibly larger scale [2]. This finding challenges conventional assumptions about AI's primary economic value and suggests that consumer applications may be equally important to professional ones.

The study concludes that ChatGPT provides significant economic value through decision support, which is especially important in knowledge-intensive jobs, but extends far beyond traditional workplace boundaries [1][3][7].

## Most Common Use Cases Identified

### The Big Three: Practical Guidance, Information Seeking, and Writing

The research identified three dominant use cases that collectively account for nearly 80% of all ChatGPT conversations [1][3][7]:

**Practical Guidance** emerges as the single most common use case, encompassing activities like tutoring and teaching, how-to advice, and ideation [2][4]. This category represents the largest share of user interactions and demonstrates ChatGPT's role as a digital advisor across various domains.

**Seeking Information** covers a broad range of queries about people, events, products, and recipes [4]. This use case positions ChatGPT as an alternative or complement to traditional search engines, offering more conversational and contextual responses.

**Writing** represents a particularly significant category, especially for work-related tasks. The research found that writing dominates work-related applications, highlighting chatbots' unique ability to generate digital outputs compared to traditional search engines [1][3][7]. Writing represents approximately 40% of work-related messages [4][8].

### Detailed Analysis of Writing Applications

Within the writing category, the researchers discovered that about two-thirds of requests involve editing, critiquing, translating, or summarizing text that users provided, rather than generating entirely new content [8]. This finding suggests that users primarily view ChatGPT as a collaborative writing assistant rather than a replacement for human creativity.

### Educational Applications

Education stands out as a notable specialized use case, with tutoring requests accounting for about 10% of all ChatGPT activity [4]. This represents a significant application area that spans both personal learning and potentially work-related skill development.

### Less Common but Notable Use Cases

The study found that computer programming and self-expression represent relatively small shares of overall usage [1][3][7], despite receiving significant attention in public discussions about AI applications. This finding helps calibrate expectations about the relative importance of different AI use cases in practice.

## Trends and Patterns Emerging from the Data

### Demographic Evolution and Accessibility

The research reveals significant demographic shifts in ChatGPT adoption over time. Early adopters were disproportionately male, but the gender gap has narrowed dramatically [1][7]. By January 2024, among users with identifiable names, 37% had typically feminine names, and by mid-2025, adoption patterns closely resembled the general adult population [3].

Geographically, the study identified higher growth rates in lower-income countries [1][7], suggesting that ChatGPT is becoming increasingly accessible across different economic contexts globally.

### Work Usage Patterns and Professional Applications

Work-related usage shows distinct patterns based on education and occupation. The research found that work usage is most common among educated users in highly-paid professional occupations [1][7]. Professionals in higher-paid technical and knowledge-driven roles are significantly more likely to employ ChatGPT at work [8].

The study utilized O*NET work activity classifications to categorize messages, mapping them to 332 Individual Work Activities (IWAs) and then aggregating to Generalized Work Activities (GWAs) [8]. This systematic approach revealed that ChatGPT applications in work contexts are heavily concentrated in knowledge work and creative tasks.

### Intensification of Usage Over Time

A critical trend identified in the research is the intensification of usage among existing users. Message volume has grown faster than user volume (5.8x versus 3.2x), indicating that ChatGPT users are engaging with the technology more intensively as they gain experience with it [5]. This suggests that familiarity breeds increased reliance rather than reduced novelty-driven usage.

### Comparative Context with Other Technologies

The research provides important context by comparing ChatGPT's growth to other major technologies. By June 2025, ChatGPT users were sending more than 2.6 billion messages per day (more than 30,000 messages per second), compared to an estimated 14 billion daily Google searches [5]. However, Google search took eight years to reach 1 billion daily searches after its 1999 public launch, while ChatGPT achieved comparable scale in a fraction of that time [5].

### Privacy-Preserving Research Methods

The study demonstrates important methodological innovations in analyzing large-scale user data while preserving privacy. The researchers used automated classifiers validated against the WildChat public dataset, with human reviewers helping to fine-tune classification prompts until achieving high fidelity with human judgment [5]. This approach enables comprehensive analysis of user behavior without compromising individual privacy.

## Implications for Understanding AI Adoption

The NBER research reveals that ChatGPT usage patterns differ significantly from initial expectations and public discourse about AI applications. The shift toward personal and home production uses, the dominance of practical guidance and information-seeking over programming and creative applications, and the rapid global diffusion across demographic groups all challenge conventional assumptions about how people integrate AI into their lives.

The study's finding that usage intensifies over time rather than declining after initial novelty suggests that ChatGPT is becoming an integral tool for many users rather than a temporary fascination. The economic implications extend far beyond workplace productivity to encompass significant value creation in personal and domestic contexts.

These findings provide crucial empirical grounding for policy discussions, business strategies, and further research into AI's societal impact. They demonstrate that understanding AI's true impact requires examining actual usage patterns rather than relying on theoretical projections or anecdotal evidence.

### Sources

[1] How People Use ChatGPT | NBER: https://www.nber.org/papers/w34255
[2] NBER WORKING PAPER SERIES HOW PEOPLE USE CHATGPT: https://www.nber.org/system/files/working_papers/w34255/w34255.pdf
[3] How people are using ChatGPT | OpenAI: https://openai.com/index/how-people-are-using-chatgpt/
[4] How People Really Use ChatGPT: Findings from NBER Research: https://techmaniacs.com/2025/09/15/how-people-really-use-chatgpt-findings-from-nber-research/
[5] How People Use ChatGPT - by David Deming - Forked Lightning: https://forklightning.substack.com/p/how-people-use-chatgpt
[6] How People Use ChatGPT: https://cdn.openai.com/pdf/a253471f-8260-40c6-a2cc-aa93fe9f142e/economic-research-chatgpt-usage-paper.pdf
[7] How People Use ChatGPT: https://www.pw.hks.harvard.edu/post/how-people-use-chatgpt
[8] How People Actually Use ChatGPT: https://medium.com/towards-artificial-intelligence/how-people-actually-use-chatgpt-2790df683c00


Research workflow completed!


## Key Takeaways

### Architecture Benefits
1. **Dynamic Decomposition** - Research structure emerges from the question, not predefined
2. **Parallel Efficiency** - Multiple researchers work simultaneously
3. **ReAct Quality** - Strategic reflection improves search decisions
4. **Scalability** - Handles token limits gracefully through compression
5. **Flexibility** - Easy to add new tools and capabilities

### When to Use This Pattern
- **Complex research questions** that need multi-angle investigation
- **Comparison tasks** where parallel research on different topics is beneficial
- **Open-ended exploration** where structure should emerge dynamically
- **Time-sensitive research** where parallel execution speeds up results

### When to Use Section-Based Instead
- **Highly structured reports** with predefined format requirements
- **Template-based content** where sections are always the same
- **Sequential dependencies** where later sections depend on earlier ones
- **Budget constraints** where token efficiency is critical

##### ✅ Answer:
##### Observations:


## Next Steps

### Extend the System
1. **Add MCP Tools** - Integrate specialized tools for your domain
2. **Custom Prompts** - Modify prompts for specific research types
3. **Different Models** - Try different Claude versions or mix models
4. **Persistence** - Use a real database for checkpointing instead of memory

### Learn More
- [LangGraph Documentation](https://langchain-ai.github.io/langgraph/)
- [Open Deep Research Repo](https://github.com/langchain-ai/open_deep_research)
- [Anthropic Claude Documentation](https://docs.anthropic.com/)
- [Tavily Search API](https://tavily.com/)

### Deploy
- Use LangGraph Cloud for production deployment
- Add proper error handling and logging
- Implement rate limiting and cost controls
- Monitor research quality and costs